In [2]:
eggnog="eggnog/test_Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.1.faa.emapper.annotations"
interpro="interpro/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.1.faa.tsv"
refseq="annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"
pannzer2="/home/davide/Desktop/GOpanzerCCMEE29.txt"

In [3]:
import pandas as pd
eggnog_df = pd.read_csv(eggnog, sep='\t', header=4,skipfooter=3, engine='python')
eggnog_df
#new dataframe with only columns #query,GOs,KEGG_ko
eggnog_df = eggnog_df[['#query', 'GOs', 'KEGG_ko']]
eggnog_df
#rename GOs to GOs_eggnog and KEGG_ko to ko_eggnog
eggnog_df = eggnog_df.rename(columns={'GOs': 'GOs_eggnog', 'KEGG_ko': 'ko_eggnog', '#query': 'protein_id'})
#set index to protein_id
eggnog_df = eggnog_df.set_index('protein_id')
eggnog_df

,GOs_eggnog,ko_eggnog
protein_id,,
WP_009632395.1,-,ko:K04751
WP_041548398.1,"GO:0003674,GO:0005488,GO:0005575,GO:0005622,GO...",ko:K02518
WP_073591590.1,-,ko:K08696
WP_190736661.1,-,-
WP_250121152.1,-,-
...,...,...
WP_275983393.1,-,-
WP_275983394.1,"GO:0003674,GO:0005488,GO:0005515,GO:0005575,GO...",ko:K02708
WP_275983395.1,-,ko:K02003


In [4]:
interpro_df = pd.read_csv(interpro, sep='\t', header=None)
interpro_df
interpro_df = interpro_df.rename(columns={0: 'protein_id', 13: 'GOs_interpro'})
#drop duplicates of protein_id
interpro_df = interpro_df.drop_duplicates(subset='protein_id')
#new dataframe with only columns #query,GOs_interpro
interpro_df = interpro_df[['protein_id', 'GOs_interpro']]

#set index to protein_id
interpro_df = interpro_df.set_index('protein_id')
interpro_df

,GOs_interpro
protein_id,
WP_250121481.1,GO:0005829(PANTHER)
WP_250124019.1,GO:0004775(PANTHER)|GO:0004776(PANTHER)|GO:000...
WP_250124642.1,-
WP_250121995.1,GO:0015979(InterPro)|GO:0015995(InterPro)|GO:0...
WP_250124302.1,-
...,...
WP_250122548.1,-
WP_250123271.1,-
WP_250125842.1,-


In [5]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
#parse refseq file
refseq_record = SeqIO.parse(refseq, "genbank")
#extract for every CDS the GO_process, GO_function and GO_component
refseq_df = pd.DataFrame(columns=['GO_process', 'GO_component', 'GO_function', 'protein_id', 'gene'])
for record in refseq_record:
    for feature in record.features:
        if feature.type == "CDS":
            #get the GO_process, GO_function and GO_component, protein_id and gene
            GO_process = feature.qualifiers.get('GO_process')
            GO_component = feature.qualifiers.get('GO_component')
            GO_function = feature.qualifiers.get('GO_function')
            protein_id = feature.qualifiers.get('protein_id', ['None'])[0]
            gene = feature.qualifiers.get('gene', ['None'])[0]
            print(gene)
            #new dataframe with protein_id as row index
            if protein_id !='None':
                data = pd.DataFrame({'GO_process': [GO_process], 'GO_component': [GO_component], 'GO_function': [GO_function], 'protein_id': [protein_id], 'gene': [gene]})
                refseq_df = pd.concat([refseq_df, data])
refseq_df = refseq_df.set_index('protein_id')


None
None
None
trpC
trpA
trpB
aroF
trpD
crtA
None
None
None
None
None
None
None
wecB
None
None
None
gndA
None
ileS
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
murA
None
None
raiA
lipB
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
thiL
None
nadD
murC
murB
None
None
None
smpB
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
cysK
None
cysH
cysC
None
None
None
None
None
None
None
None
grxD
None
None
None
tadA
grxC
glpX
None
surE
None
None
None
None
None
None
None
None
None
None
None
None
pgeF
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
yidD
None
None
None
None
None
None
None
None
None
None
None
None
minE
minD
minC
None
None
None
None
None
None
dapF
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [6]:
refseq_df

,GO_process,GO_component,GO_function,gene
protein_id,,,,
WP_275983370.1,None,None,None,None
WP_250123595.1,None,None,None,None
WP_250123596.1,[GO:0000162 - tryptophan biosynthetic process ...,None,[GO:0004049 - anthranilate synthase activity [...,None
WP_250123597.1,[GO:0006568 - tryptophan metabolic process [Ev...,None,[GO:0004425 - indole-3-glycerol-phosphate synt...,trpC
WP_250123598.1,[GO:0000162 - tryptophan biosynthetic process ...,None,[GO:0004834 - tryptophan synthase activity [Ev...,trpA
...,...,...,...,...
WP_256479003.1,None,None,[GO:0004803 - transposase activity [Evidence I...,None
WP_250126520.1,None,None,None,None
WP_250126521.1,None,None,None,None


In [7]:
blastko = pd.read_csv('/home/davide/Downloads/user_ko.txt', sep='\t', header=None)
blastko = blastko.rename(columns={0: 'protein_id', 1: 'ko_blastkoala'})
blastko = blastko.set_index('protein_id')
blastko

,ko_blastkoala
protein_id,
WP_009632395.1,K04751
WP_041548398.1,K02518
WP_073591590.1,K08696
WP_190736661.1,K00355
WP_250121152.1,NaN
...,...
WP_275983394.1,K02708
WP_275983395.1,K02003
WP_275983396.1,K21498


In [37]:
pannzer2_df = pd.read_csv(pannzer2, sep='\t', index_col=0, dtype={'goid': str})
pannzer2_df
#made one with ponly rows with BP in ontology column
pannzer2_df_BP = pannzer2_df[pannzer2_df['ontology'] == 'BP']
pannzer2_df_BP = pannzer2_df_BP.rename(columns={'goid': 'GO_process_pannzer2'})
pannzer2_df_BP
#rename index to protein_id
pannzer2_df_BP = pannzer2_df_BP.rename_axis('protein_id')
pannzer2_df_BP
#drop second column and 5 to last
pannzer2_df_BP = pannzer2_df_BP.drop(pannzer2_df_BP.columns[0], axis=1)
pannzer2_df_BP = pannzer2_df_BP.drop(pannzer2_df_BP.columns[2:], axis=1)
pannzer2_df_BP
#join first two columns as one
pannzer2_df_BP['GO_process_pannzer2'] = 'GO:' + pannzer2_df_BP['GO_process_pannzer2'] + ' - ' + pannzer2_df_BP['desc']
#drop columns desc
pannzer2_df_BP = pannzer2_df_BP.drop('desc', axis=1)
pannzer2_df_BP 


,GO_process_pannzer2
protein_id,
WP_009632395.1,GO:0006808 - regulation of nitrogen utilization
WP_041548398.1,GO:0006413 - translational initiation
WP_250121155.1,GO:0009772 - photosynthetic electron transport...
WP_250121157.1,GO:0055085 - transmembrane transport
WP_250121157.1,GO:0006826 - iron ion transport
...,...
WP_275983383.1,GO:0019646 - aerobic electron transport chain
WP_275983383.1,GO:1902600 - proton transmembrane transport
WP_275983386.1,GO:0009231 - riboflavin biosynthetic process


In [45]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open("/home/davide/Downloads/KEGGbriteCCMEE29.html"), 'html.parser')
text=soup.get_text()
text=text[text.find("ko01001"):]
lines = text.split("\n")
diz={}

for i,line in enumerate(lines):
    if line.startswith("ko"):
        current_key = line[:line.index("(")].strip()
        diz[current_key] = []
    elif line.startswith("WP"):
        proteins = line.split(",") if "," in line else [line]
        diz[current_key].extend(proteins)
from pprint import pprint

import pandas as pd
all_proteins = set(protein for proteins in diz.values() for protein in proteins)
dfs = []
for protein in all_proteins:
    data = {pathway: 1 if protein in proteins else 0 for pathway, proteins in diz.items()}
    df = pd.DataFrame(data, index=[protein])
    dfs.append(df)
result = pd.concat(dfs)
result["KEGG_brite"] = result.apply(lambda row: ", ".join(row[row == 1].index), axis=1)
result.drop(result.columns[:-1], axis=1, inplace=True)
df_brite = result

In [48]:
with open("/home/davide/Downloads/KEGGCCMEE.html") as f:
    lines = f.readlines()

#use beautifulsoup to parse the html
from bs4 import BeautifulSoup
soup = BeautifulSoup(open("/home/davide/Downloads/KEGGCCMEE.html"), 'html.parser')
#find all a tags and all b tags
text=soup.get_text()
text=text[:text.find("Organismal Systems")]
lines = text.split("\n")
diz = {}
for i,line in enumerate(lines):
    if line.startswith("0"):
        if "(" in line:
            current_key = line[:line.index("(")].strip()
        else:
            current_key = line
            #add to key lines[i+j] until lines[i+j] starts has (
            j = 1
            while not " (" in lines[i+j]:
                current_key += lines[i+j]
                j += 1
        diz[current_key] = []

    elif line.startswith("WP_"):
        proteins = line.split(", ") if ", " in line else [line]
        diz[current_key].extend(proteins)
#now make a dataframe that for every value of the dictionary has a row with the key as index and the keys of the dictionary in which appears as columns
import pandas as pd

# Get a list of all unique protein IDs
all_proteins = set(protein for proteins in diz.values() for protein in proteins)

# Initialize an empty list to hold the dataframes
dfs = []

# Iterate over each protein in the set of all proteins
for protein in all_proteins:
    # Create a dictionary where the keys are the pathways and the values are 1 if the protein is present in the pathway and 0 otherwise
    data = {pathway: 1 if protein in proteins else 0 for pathway, proteins in diz.items()}
    # Convert the dictionary into a DataFrame and append it to the list
    df = pd.DataFrame(data, index=[protein])
    dfs.append(df)

# Concatenate all the dataframes into a single dataframe
result = pd.concat(dfs)
result["KEGG_pathways"] = result.apply(lambda row: ", ".join(row[row == 1].index), axis=1)
result.drop(result.columns[:-1], axis=1, inplace=True)
df_pathways = result


In [49]:
df = refseq_df.join(eggnog_df, how='outer')
df = df.join(interpro_df, how='outer')
df = df.join(blastko, how='outer')
df
#reored columns
df = df[['gene', 'GO_process', 'GO_component', 'GO_function', 'GOs_eggnog', 'GOs_interpro', 'ko_eggnog', 'ko_blastkoala']]
df=df.join(pannzer2_df_BP, how='outer')
df = df.join(df_brite, how='outer')
df = df.join(df_pathways, how='outer')
#reorder for gene column reverse order
df = df.sort_values(by='gene', ascending=False)
df

,gene,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_brite,KEGG_pathways
protein_id,,,,,,,,,,,
WP_250124752.1,zwf,[GO:0006098 - pentose-phosphate shunt [Evidenc...,None,[GO:0004345 - glucose-6-phosphate dehydrogenas...,-,GO:0004345(PANTHER)|GO:0005829(PANTHER)|GO:000...,ko:K00036,K00036,GO:0006098 - pentose-phosphate shunt,ko04147 Exosome,"01100 Metabolic pathways, 01110 Biosynthesis o..."
WP_250124752.1,zwf,[GO:0006098 - pentose-phosphate shunt [Evidenc...,None,[GO:0004345 - glucose-6-phosphate dehydrogenas...,-,GO:0004345(PANTHER)|GO:0005829(PANTHER)|GO:000...,ko:K00036,K00036,GO:0006006 - glucose metabolic process,ko04147 Exosome,"01100 Metabolic pathways, 01110 Biosynthesis o..."
WP_250123417.1,zds,None,None,None,-,GO:0016491(PANTHER),ko:K00514,K00514,GO:0016117 - carotenoid biosynthetic process,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o..."
WP_250121918.1,yqeK,None,None,None,-,-,-,NaN,NaN,NaN,NaN
WP_250124094.1,ylqF,[GO:0042254 - ribosome biogenesis [Evidence IEA]],[GO:0005737 - cytoplasm [Evidence IEA]],"[GO:0003924 - GTPase activity [Evidence IEA], ...",-,-,ko:K14540,K14540,NaN,ko03009 Ribosome biogenesis,NaN
...,...,...,...,...,...,...,...,...,...,...,...
WP_250123219.1,None,None,None,None,-,-,-,NaN,NaN,NaN,NaN
WP_250123218.1,None,None,None,None,-,GO:0003824(InterPro)|GO:0051536(InterPro),-,NaN,NaN,NaN,NaN
WP_250123216.1,None,None,None,None,-,GO:0005887(PANTHER),ko:K03589,K03589,NaN,ko03036 Chromosome and associated proteins,04112 Cell cycle - Caulobacter


In [50]:
df.to_csv('/home/davide/Desktop/annotazioniGOkoCCMEE29.csv')

In [10]:
import networkx
import obonet

# Load the GO graph
url = "/home/davide/Downloads/go-basic.obo"
graph = obonet.read_obo(url)

# Create a dictionary with id as key and name and namespace as value
dict = {id_: (data.get('name'), data.get('namespace')) 
                          for id_, data in graph.nodes(data=True)}

names="""GO:0003723(InterPro)|GO:0003743(InterPro)|GO:0006413(InterPro)"""
names = names.split('|')
names = [name.split('(')[0] for name in names]
for name in names:
    print(name, dict.get(name, ['Not found']))
print(dict.get('GO:0006313', ['Not found']))


GO:0003723 ('RNA binding', 'molecular_function')
GO:0003743 ('translation initiation factor activity', 'molecular_function')
GO:0006413 ('translational initiation', 'biological_process')
('DNA transposition', 'biological_process')
